In [49]:
import os
import pandas as pd
import nltk

In [50]:
print os.getcwd()

/Users/palermospenano/Desktop/github_proj/ejmr_analysis


In [87]:
ejmr_df = pd.read_csv('./ejmr_data/ejmr_data_cleand.csv')
# ejmr_df.head()

In [88]:
# word frequency (reference: https://stackoverflow.com/questions/40206249/count-of-most-popular-words-in-a-pandas-dataframe)
ejmr_df['cleanTitle'] = ejmr_df['cleanTitle'].str.replace(',', '')  ## returns a pandas series
ejmr_df.head()

,pandas_index,post_title,num_posts,views,votes,how_fresh,download_time,title,post_link,freshness,cleanTitle
0,0,Pls explain me how machines can learn?,5,95,0-0,1 day,2017-06-11 03:37:22.777761,"<a href=""https://www.econjobrumors.com/topic/p...",https://www.econjobrumors.com/topic/pls-explai...,"<a href=""https://www.econjobrumors.com/topic/p...",pls explain machines learn
1,1,Machine learning is Satan worshipping in disguise,17,337,6-0,1 day,2017-06-11 03:37:22.777852,"<a href=""https://www.econjobrumors.com/topic/m...",https://www.econjobrumors.com/topic/machine-le...,"<a href=""https://www.econjobrumors.com/topic/m...",machine learning satan worshipping disguise
2,2,Should I buy a BMW,29,960,0-0,1 day,2017-06-11 03:37:22.777956,"<a href=""https://www.econjobrumors.com/topic/s...",https://www.econjobrumors.com/topic/should-i-b...,"<a href=""https://www.econjobrumors.com/topic/s...",buy bmw
3,3,A proposal on how to replace student evaluations,22,561,4-3,1 day,2017-06-11 03:37:22.778058,"<a href=""https://www.econjobrumors.com/topic/a...",https://www.econjobrumors.com/topic/a-proposal...,"<a href=""https://www.econjobrumors.com/topic/a...",proposal replace student evaluations
4,4,"Roman Catholics, put down your rosaries and ex...",1,69,1-0,1 day,2017-06-11 03:37:22.778133,"<a href=""https://www.econjobrumors.com/topic/r...",https://www.econjobrumors.com/topic/roman-cath...,"<a href=""https://www.econjobrumors.com/topic/r...",roman catholics put rosaries explain


In [90]:
txt = ejmr_df['cleanTitle'].str.cat(sep=' ')

In [91]:
words = nltk.tokenize.word_tokenize(txt)
word_dist = nltk.FreqDist(txt)

In [92]:
words

['pls',
 'explain',
 'machines',
 'learn',
 'machine',
 'learning',
 'satan',
 'worshipping',
 'disguise',
 'buy',
 'bmw',
 'proposal',
 'replace',
 'student',
 'evaluations',
 'roman',
 'catholics',
 'put',
 'rosaries',
 'explain',
 'eh',
 'eh',
 'brains',
 'aarghhh',
 'brains',
 'taylor',
 'swift',
 'puts',
 'music',
 'spotify',
 'katy',
 'perry',
 'release',
 'date',
 '2016',
 'maryland',
 'senior',
 'hires',
 'seminar',
 'notice',
 'ccer',
 'ejmr',
 'rule',
 '1',
 'respectful',
 'nudes',
 'cradle',
 'suplexes',
 'left',
 'academia',
 'current',
 'jfe',
 'editor',
 'participation',
 'rates',
 'phone',
 'app',
 'ideas',
 'econ',
 'vs',
 'fen',
 'chat',
 'steve',
 'davies',
 'uk',
 'politics',
 'mr',
 'ndb',
 'new',
 'development',
 'bank',
 'junior',
 'young',
 'professionals',
 'thred',
 'conference',
 'share',
 'math',
 'stat',
 'phd',
 'transfers',
 'cohort',
 'states',
 'moving',
 'green',
 'energy',
 'quickly',
 'mr',
 'qatar',
 'cut',
 'extreme',
 'sudden',
 'mr',
 'acceptable'